<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/135_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Earth Engine Image Segmentation with the Segment Anything Model**

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [16]:
 !pip install -U geemap

In [17]:
%pip install segment-geospatial pycrs

In [18]:
import ee
import geemap
from samgeo import SamGeo

In [19]:
ee.Initialize(project='ee-amalmatoussi18')

In [20]:
Map = geemap.Map()
point = ee.Geometry.Point(-122.259679, 37.871838)
collection = (
    ee.ImageCollection("USDA/NAIP/DOQQ")
    .filterBounds(point)
    .filterDate("2008-01-01", "2018-01-01")
    .filter(ee.Filter.listContains("system:band_names", "N"))
)
image = collection.first()
Map.addLayer(image, {}, "NAIP")
Map.centerObject(point, 16)
Map

Map(center=[37.87183799999999, -122.259679], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [21]:
bbox = Map.user_roi_coords()
if bbox is None:
    bbox = [-122.2666, 37.8682, -122.252, 37.8752]

In [22]:
geemap.ee_to_geotiff(
    image, "naip.tif", bbox, zoom=17, vis_params={"bands": ["R", "G", "B"]}
)

Downloaded image 1/30
Downloaded image 2/30
Downloaded image 3/30
Downloaded image 4/30
Downloaded image 5/30
Downloaded image 6/30
Downloaded image 7/30
Downloaded image 8/30
Downloaded image 9/30
Downloaded image 10/30
Downloaded image 11/30
Downloaded image 12/30
Downloaded image 13/30
Downloaded image 14/30
Downloaded image 15/30
Downloaded image 16/30
Downloaded image 17/30
Downloaded image 18/30
Downloaded image 19/30
Downloaded image 20/30
Downloaded image 21/30
Downloaded image 22/30
Downloaded image 23/30
Downloaded image 24/30
Downloaded image 25/30
Downloaded image 26/30
Downloaded image 27/30
Downloaded image 28/30
Downloaded image 29/30
Downloaded image 30/30
Saving GeoTIFF. Please wait...
Image saved to naip.tif


In [23]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    device=None,
    sam_kwargs=None,
)

In [ ]:
sam.generate("naip.tif", output="masks.tif", foreground=True, unique=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

In [ ]:
Map.add_raster("annotations.tif", opacity=0.5, layer_name="Masks")
Map

In [ ]:
sam.tiff_to_vector("masks.tif", "masks.shp")

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
Map.add_vector("masks.shp", layer_name="Vector", style=style)
Map